# Business Logic
---
This is going to be an experimentation for natural language processing in the context of finance. Using libraries like FinBERT, and LDA, we shall leverage language models to help us inform people about what's going in the world of a specific company. Practices will be followed by extracting the article for a given stock and reusing
the methodddology for analysis here.
---


# Importing Libraries

In [3]:
!pip install feedparser

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 2.5 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=ecd00de776f6874a22c5c548c4e9ee795f061726c417779e60ae6cab3ce5a3d1
  Stored in directory: /root/.cache/pip/wheels/03/f5/1a/23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built sgmllib3k


In [ ]:
import yfinance as yf
import pandas as pd
from transformers import BertForSequenceClassification, BertTokenizer
import feedparser
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import re

In [4]:
class YahooFinanceFullArticleScraper:
    """
    Extracts full article content from Yahoo Finance RSS feeds
    Uses RSS for article discovery, then fetches full content from article URLs
    """

    def __init__(self):
        self.base_rss_url = "https://feeds.finance.yahoo.com/rss/2.0/headline?s={}&region=US&lang=en-US"
        self.headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}

    def _extract_article_text(self, url):
        """Extract full article text from a Yahoo Finance article URL"""
        try:
            response = requests.get(url, headers=self.headers, timeout=10)
            response.raise_for_status()

            soup = BeautifulSoup(response.content, 'html.parser')

            # Find article content - Yahoo Finance uses various selectors
            article_content = None

            # Try common article content selectors
            selectors = [
                'article',
                '[data-module="ArticleBody"]',
                '.caas-body',
                '.article-body',
                '[class*="article"]',
                '[class*="content"]'
            ]

            for selector in selectors:
                article_content = soup.select_one(selector)
                if article_content:
                    break

            if not article_content:
                # Fallback: find main content area
                article_content = soup.find('main') or soup.find('article')

            if article_content:
                # Remove script and style elements
                for script in article_content(["script", "style", "nav", "footer", "header"]):
                    script.decompose()

                # Extract text and clean it
                text = article_content.get_text(separator=' ', strip=True)
                # Clean up multiple whitespaces
                text = re.sub(r'\s+', ' ', text).strip()
                return text

            return None

        except Exception as e:
            return None

    def get_full_articles_for_ticker(self, ticker, max_articles=10, verbose=False):
        """
        Get full article content for a ticker symbol

        Args:
            ticker (str): Stock ticker symbol
            max_articles (int): Maximum number of articles to fetch
            verbose (bool): Print progress messages

        Returns:
            list: List of articles with full text content
        """
        articles = []

        if verbose:
            print(f"Fetching RSS feed for {ticker}...")

        try:
            # Get RSS feed
            feed_url = self.base_rss_url.format(ticker.upper())
            feed = feedparser.parse(feed_url)

            if not feed.entries:
                if verbose:
                    print(f"  ⚠ No articles found for {ticker}")
                return articles

            if verbose:
                print(f"Found {len(feed.entries)} articles in RSS feed")
                print(f"Fetching full content (filtering for articles >150 words)..\n")

            # Process articles until we have max_articles that meet the word count requirement
            articles_processed = 0
            articles_skipped = 0

            for entry in feed.entries:
                # Stop if we have enough articles
                if len(articles) >= max_articles:
                    break

                articles_processed += 1
                article_url = entry.get('link', '').strip()
                title = entry.get('title', '').strip()

                if not article_url:
                    continue

                if verbose:
                    title_short = title[:60] + "..." if len(title) > 60 else title
                    print(f"[{articles_processed}] Fetching: {title_short}...")

                # Extract full article text
                full_text = self._extract_article_text(article_url)

                # Parse publication date
                published = entry.get('published', '')
                published_datetime = None
                if hasattr(entry, 'published_parsed') and entry.published_parsed:
                    try:
                        published_datetime = datetime(*entry.published_parsed[:6])
                    except:
                        pass

                # Calculate word count
                word_count = len(full_text.split()) if full_text else 0

                # Filter: Only keep articles with more than 150 words
                if word_count <= 150:
                    articles_skipped += 1
                    if verbose:
                        print(f"    ⚠ Skipped: {word_count} words (minimum 150 required)")
                    continue

                article = {
                    'ticker': ticker.upper(),
                    'title': title,
                    'link': article_url,
                    'rss_description': entry.get('summary', '').strip(),
                    'published': published,
                    'published_datetime': published_datetime,
                    'guid': entry.get('guid', ''),
                    'full_text': full_text or '',
                    'word_count': word_count,
                    'has_full_text': full_text is not None and len(full_text) > 0
                }

                articles.append(article)

                if verbose and full_text:
                    print(f"    ✓ Retrieved {word_count} words")
                elif verbose:
                    print(f"    ⚠ Could not extract content")

            # Summary
            if verbose:
                print(f"\n📊 Summary:")
                print(f"   Articles processed: {articles_processed}")
                print(f"   Articles skipped (<150 words): {articles_skipped}")
                print(f"   Articles returned: {len(articles)}")

            return articles

        except Exception as e:
            if verbose:
                print(f"  ✗ Error: {e}")
            return articles

# Initialize scraper
scraper = YahooFinanceFullArticleScraper()

# Get full articles for a ticker (e.g., 'AAPL')
# The 'articles' variable will contain a list of dictionaries, each with full article content
articles_for_nlp = scraper.get_full_articles_for_ticker('AAPL', max_articles=5, verbose=False)

Successfully fetched 5 articles for NLP analysis.
First article title: Apple Stock Heads for Record as Big Tech Pecking Order Gets Reshuffled
First article full text preview: Apple Stock Heads for Record as Big Tech Pecking Order Gets Reshuffled Apple Stock Heads for Record as Big Tech Pecking Order Gets Reshuffled · Barrons.com Martin Baccardax Wed, November 26, 2025 at 4...


In [6]:
articles_for_nlp[0]['full_text']

'Apple Stock Heads for Record as Big Tech Pecking Order Gets Reshuffled Apple Stock Heads for Record as Big Tech Pecking Order Gets Reshuffled · Barrons.com Martin Baccardax Wed, November 26, 2025 at 4:07 PM EST 4 min read AVGO NVDA GOOG AAPL META It may be too early to declare a new sheriff in residence, but Google parent Alphabet emergence as the hottest new thing in artificial intelligence has triggered a massive realignment among the world’s biggest tech stocks. Google is set to open at a fresh record high on Wednesday, with a market value of just under $4 trillion, to extend its November advance to around 17%. On the downside, Nvidia stock is set to open more than 15% south of its last all-time peak, as it slowly careens through correction territory heading into the final month of the year. Continue Reading View comments Terms and Privacy Policy Privacy Dashboard More Info'

In [7]:
articles_for_nlp[4]['full_text']

'A comprehensive list of 2025 tech layoffs Cody Corrall, Alyssa Stringer, Kate Park Wed, November 26, 2025 at 2:28 PM EST 29 min read HPQ AMZN PLTK RIVN GOOG Image Credits:Bryce Durbin / TechCrunch The tech layoff wave is still kicking in 2025. Last year saw more than 150,000 job cuts across 549 companies, according to independent layoffs tracker Layoffs.fyi . So far this year, more than 22,000 workers have been the victim of reductions across the tech industry, with a staggering 16,084 cuts taking place in February alone. We’re tracking layoffs in the tech industry in 2025 so you can see the trajectory of the cutbacks and understand the impact on innovation across all types of companies. As businesses continue to embrace AI and automation, this tracker serves as a reminder of the human impact of layoffs — and what could be at stake with increased innovation. Below you’ll find a comprehensive list of all the known tech layoffs that have occurred in 2025, which will be updated regularly